In [7]:
   from pyngrok import ngrok

   tunnels = ngrok.get_tunnels()
   for tunnel in tunnels:
       print(tunnel.public_url)

tcp://0.tcp.ngrok.io:10315
tcp://0.tcp.ngrok.io:12245
https://dee2-34-168-214-241.ngrok-free.app


In [10]:
   for tunnel in tunnels:
       ngrok.disconnect(tunnel.public_url)

In [9]:
# Install necessary packages
!pip install pyngrok flask_sqlalchemy

# Import necessary libraries
import getpass
import os
import threading
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from pyngrok import ngrok, conf

# Prompt for ngrok authtoken
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

# List and close existing tunnels
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

# Open a TCP ngrok tunnel to the SSH server
try:
    connection_string = ngrok.connect("22", "tcp").public_url
    ssh_url, port = connection_string.strip("tcp://").split(":")
    print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")
except Exception as e:
    print(f"Failed to open SSH tunnel: {e}")

# Initialize Flask app
app = Flask(__name__)
port = 5010

# Configure database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///crud.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Define a model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)

# Create the database
with app.app_context():
    db.create_all()

# Open a ngrok tunnel to the HTTP server
try:
    public_url = ngrok.connect(port).public_url
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
except Exception as e:
    print(f"Failed to open HTTP tunnel: {e}")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define Flask routes for CRUD operations
@app.route("/users", methods=["GET"])
def get_users():
    users = User.query.all()
    return jsonify([{"id": user.id, "name": user.name} for user in users])

@app.route("/users", methods=["POST"])
def add_user():
    data = request.get_json()
    name = data.get("name")
    if name:
        new_user = User(name=name)
        db.session.add(new_user)
        db.session.commit()
        return jsonify({"id": new_user.id, "name": new_user.name}), 201
    return jsonify({"error": "Name is required"}), 400

@app.route("/users/<int:user_id>", methods=["PUT"])
def update_user(user_id):
    data = request.get_json()
    user = User.query.get(user_id)
    if user:
        user.name = data.get("name", user.name)
        db.session.commit()
        return jsonify({"id": user.id, "name": user.name})
    return jsonify({"error": "User not found"}), 404

@app.route("/users/<int:user_id>", methods=["DELETE"])
def delete_user(user_id):
    user = User.query.get(user_id)
    if user:
        db.session.delete(user)
        db.session.commit()
        return jsonify({"message": "User deleted"})
    return jsonify({"error": "User not found"}), 404

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False, "port": port}).start()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


 * ngrok tunnel available, access with `ssh root@8.tcp.ngrok.io -p18745`
 * ngrok tunnel "https://2441-34-168-214-241.ngrok-free.app" -> "http://127.0.0.1:5010"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5010
INFO:werkzeug:Press CTRL+C to quit
